<a href="https://colab.research.google.com/github/mohitsriv23/Neptune-AI/blob/main/Neptune_Yolo_by_Mohit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# Download dataset
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("myworkspace-bliu9").project("white-nail-classifier-2")
version = project.version(1)
dataset = version.download("folder")


/content/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to White-Nail-Classifier-2-1 in folder:: 100%|██████████| 2155/2155 [00:01<00:00, 1727.12it/s]


In [ ]:
from getpass import getpass


my_api_token = getpass("Enter your Neptune API token: ")
print(my_api_token)

Enter your Neptune API token: ··········
eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiYmZjZDM2OC02ZmU0LTQ2YmItOGEyNy00NmExYjlmYjcwMzAifQ==


In [ ]:
import os
os.environ["NEPTUNE_API_TOKEN"] = my_api_token
os.environ["NEPTUNE_PROJECT"] = "tech.licenses/VitaminDifficeincy"

In [6]:
import os
import pandas as pd
import neptune

# Initialize Neptune run
run = neptune.init_run(
    project=os.environ.get("NEPTUNE_PROJECT"),
    api_token=my_api_token,
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)

%cd {HOME}
print(dataset.location)
!yolo task=classify mode=train model=yolov8s-cls.pt data={dataset.location} epochs=300 imgsz=320 batch=16

results_csv = os.path.join(HOME, "/content/runs/classify/train", "results.csv")
results = pd.read_csv(results_csv)

for _, row in results.iterrows():
    # run["train/loss"].log(row["train/loss"])
    # run["metrics/accuracy_top1"].log(row["metrics/accuracy_top1"])
    # run["epoch"].log(row["epoch"])
    run["train/loss"].append(row["train/loss"])
    run["metrics/accuracy_top1"].append(row["metrics/accuracy_top1"])
    run["epoch"].append(row["epoch"])

# Save and upload the model to Neptune
model_path = os.path.join(HOME, "runs/classify/train", "weights", "best.pt")
if os.path.exists(model_path):
    run["model"].upload(model_path)
else:
    print("Model file not found!")

run.stop()

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/tech.licenses/VitaminDifficeincy/e/VITD-37
/content
/content/datasets/White-Nail-Classifier-2-1
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 12.3M/12.3M [00:00<00:00, 94.7MB/s]
Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/datasets/White-Nail-Classifier-2-1, epochs=300, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_l

In [ ]:
results_csv = os.path.join(HOME, "/content/runs/classify/train", "results.csv")
results = pd.read_csv(results_csv)

for _, row in results.iterrows():
    # run["train/loss"].log(row["train/loss"])
    # run["metrics/accuracy_top1"].log(row["metrics/accuracy_top1"])
    # run["epoch"].log(row["epoch"])
    run["train/loss"].append(row["train/loss"])
    run["metrics/accuracy_top1"].append(row["metrics/accuracy_top1"])
    run["epoch"].append(row["epoch"])

# Save and upload the model to Neptune
model_path = os.path.join(HOME, "runs/classify/train", "weights", "best.pt")
if os.path.exists(model_path):
    run["model"].upload(model_path)
else:
    print("Model file not found!")

run.stop()


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 606 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 606 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/tech.licenses/VitaminDifficeincy/e/VITD-34/metadata


In [ ]:
# %cd {HOME}
# print(dataset.location)
# !yolo task=classify mode=train model=yolov8s-cls.pt data={dataset.location} epochs=2 imgsz=320 batch=16

/content
/content/datasets/Beaus-Lines-Nails-3
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 12.3M/12.3M [00:00<00:00, 90.7MB/s]
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/datasets/Beaus-Lines-Nails-3, epochs=2, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_rat

In [ ]:
# import os
# from ultralytics import YOLO
# import neptune
# import time

# # Initialize Neptune run
# run = neptune.init_run(
#     project=os.environ.get("NEPTUNE_PROJECT"),
#     api_token=my_api_token,
#     capture_hardware_metrics=True,
#     capture_stderr=True,
#     capture_stdout=True,
# )

# # Define model name
# MODEL_NAME = "beau.pt"

# # Load the YOLOv8 classification model
# model = YOLO('yolov8s-cls.pt')

# # Log model details to Neptune
# run["model/name"] = MODEL_NAME
# run["model/task"] = "classification"

# # Training parameters
# data = dataset.location
# epochs = 200
# batch_size = 16
# imgsz = 320
# device = "0"

# # Training loop
# for epoch in range(epochs):
#     print(f"Starting epoch {epoch + 1}/{epochs}...")
#     start_time = time.time()

#     # Train for one epoch and capture results
#     results = model.train(
#         data=data,
#         epochs=1,  # Train for one epoch at a time to log metrics
#         imgsz=imgsz,
#         batch=batch_size,
#         device=device,
#         verbose=False  # Suppress verbose logs
#     )

#     # Extract metrics from results
#     metrics = results.results_dict
#     # print(metrics)
#     train_acc1 = metrics.get("metrics/accuracy_top1", None)
#     train_acc5 = metrics.get("metrics/accuracy_top5", None)

#     # Append metrics to Neptune
#     if train_acc1 is not None:
#         run["train/accuracy1"].append(train_acc1)
#     if train_acc5 is not None:
#         run["train/accuracy5"].append(train_acc5)

#     # Log time taken for the epoch
#     # run["train/epoch_time"].append(time.time() - start_time)


# # Save the trained model
# model_path = "/content/beau_nail_model.pt"
# model.save(model_path)
# run["model/saved_path"] = model_path

# # Stop Neptune run
# run.stop()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/tech.licenses/VitaminDifficeincy/e/VITD-22


100%|██████████| 12.3M/12.3M [00:00<00:00, 83.5MB/s]


Starting epoch 1/200...
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/datasets/Beaus-Lines-Nails-3, epochs=1, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


WARNING ⚠️ NeptuneAI installed but not initialized correctly, not logging this run. No existing Project was found at 

----NeptuneProjectNotFoundException------------------------------------

We couldn't find project "Ultralytics".

Did you mean any of these?
    - tech.licenses/VitaminDifficeincy

You can check all of your projects on the Projects page:
    - https://app.neptune.ai/tech.licenses/-/projects

You may want to check the following docs page:
    - https://docs.neptune.ai/setup/creating_project/

Need help?-> https://docs.neptune.ai/getting_help
.
TensorBoard: Start with 'tensorboard --logdir runs/classify/train', view at http://localhost:6006/
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all      0.936          1
Speed: 0.4ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train22222222222222222222222222222222222222222222222222
Starting epoch 52/200...
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/datasets/Beaus-Lines-Nails-3, epochs=1, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train222222222222222222222222222222222222222222222222222, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, ha

train: Scanning /content/datasets/Beaus-Lines-Nails-3/train... 810 images, 0 corrupt: 100%|██████████| 810/810 [00:00<?, ?it/s]
val: Scanning /content/datasets/Beaus-Lines-Nails-3/val... 235 images, 0 corrupt: 100%|██████████| 235/235 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/train222222222222222222222222222222222222222222222222222
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


        1/1     0.887G          0         16        320:  25%|██▌       | 13/51 [00:03<00:10,  3.68it/s]


KeyboardInterrupt: 